# **Column Transformer**

In [157]:
import pandas as pd 
import numpy as np

In [158]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

In [159]:
df = pd.read_csv('./dataset/covid_toy.csv')

In [160]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [161]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [162]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2, random_state=42)

In [163]:
X_train

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
88,5,Female,100.0,Mild,Kolkata
26,19,Female,100.0,Mild,Kolkata
42,27,Male,100.0,Mild,Delhi
69,73,Female,103.0,Mild,Delhi
...,...,...,...,...,...
60,24,Female,102.0,Strong,Bangalore
71,75,Female,104.0,Strong,Delhi
14,51,Male,104.0,Mild,Bangalore
92,82,Female,102.0,Strong,Kolkata


In [164]:
y_train

55    Yes
88     No
26    Yes
42    Yes
69     No
     ... 
60    Yes
71     No
14     No
92     No
51    Yes
Name: has_covid, Length: 80, dtype: object

## Simple way

In [165]:
# adding simple imputer to fever col
imputer = SimpleImputer()
X_train_fever = imputer.fit_transform(X_train[['fever']])
# also with test data
X_test_fever = imputer.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [166]:
# ordinal encoding to cough col 
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [167]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [168]:
# extracting Age
X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values

# also with test data
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

X_train_age.shape

(80, 1)

In [169]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)
# also with test data
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

X_train_transformed.shape

(80, 7)

# **By using ColumnTransformer**

In [170]:
from sklearn.compose import ColumnTransformer

In [171]:
transformer_X = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')
# transformer_y = ColumnTransformer(transformers=[
#     ('tnf1', LabelEncoder(), ['has_covid'])
# ], remainder='drop')

In [172]:
transformed_X_train = transformer_X.fit_transform(X_train)

In [173]:
transformed_X_test = transformer_X.transform(X_test)

In [174]:
X_train.columns.tolist()

['age', 'gender', 'fever', 'cough', 'city']

In [175]:
le = LabelEncoder()
le.fit(y_train)
transformed_y_train = le.transform(y_train)
transformed_y_test = le.transform(y_test)

In [176]:
transformed_y_test

array([0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0])

In [177]:
# Train the model
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(transformed_X_train, transformed_y_train)

DecisionTreeClassifier()

In [178]:
y_pred = model.predict(transformed_X_test)

In [179]:
y_pred

array([0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0])

In [180]:
transformed_y_test

array([0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0])

In [181]:
from sklearn.metrics import classification_report
print(classification_report(transformed_y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.38      0.50        13
           1       0.38      0.71      0.50         7

    accuracy                           0.50        20
   macro avg       0.55      0.55      0.50        20
weighted avg       0.60      0.50      0.50        20

